In [0]:
spark.conf.set(
    "fs.azure.account.key.projectadf1.dfs.core.windows.net",
    "CSiXNWhibZG0fqyBMd049UrUimppMcGnXCzGUICGV/PCtQ5MAfQ==")

In [0]:
from pyspark.sql.functions import col, trim, round, current_timestamp, when
from pyspark.sql.types import DoubleType

billing_raw_df = spark.read.table("healthcare_raw.billing")

In [0]:
from pyspark.sql.functions import lower
billing_cleaned_df = (billing_raw_df.select(col("InvoiceID").alias("invoice_id"),
                                 col("PatientID").alias("patient_id"),
                                 col("Items").alias("items"),
                                 col("Amount").alias("amount")))
billing_cleaned_df.display()

invoice_id,patient_id,items,amount
null,894,Immunizations,956065.0
null,448,Cataract surgery,188997.0
null,618,Pediatric surgery,524091.0
null,117,Rhinoplasty,901393.0
null,374,Antibiotic therapy,448957.0
null,852,Emergency surgical procedures,869041.0
null,834,Pediatric surgery,492022.0
null,140,Hemodynamic monitoring,644410.0
null,459,Emergency surgical procedures,295907.0
null,734,Immunotherapy (allergy shots),766908.0


In [0]:
(billing_cleaned_df.write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("healthcare_processed.billing"))

# Write to cloud storage
billing_cleaned_df.write.mode("overwrite").parquet(f"abfss://source@projectadf1.dfs.core.windows.net/billing")


In [0]:
from pyspark.sql.functions import sum, count,avg

summary_df = billing_cleaned_df.agg(
    count("*").alias("total_invoices"),
    sum("amount").alias("total_revenue"),
    round(avg("amount"), 2).alias("average_invoice_amount")
)

print("Billing Summary:")
display(summary_df)

Billing Summary:


total_invoices,total_revenue,average_invoice_amount
1000,5.10275708E8,510275.71


In [0]:
%sql
select * from healthcare_processed.billing

invoice_id,patient_id,items,amount
null,894,Immunizations,956065.0
null,448,Cataract surgery,188997.0
null,618,Pediatric surgery,524091.0
null,117,Rhinoplasty,901393.0
null,374,Antibiotic therapy,448957.0
null,852,Emergency surgical procedures,869041.0
null,834,Pediatric surgery,492022.0
null,140,Hemodynamic monitoring,644410.0
null,459,Emergency surgical procedures,295907.0
null,734,Immunotherapy (allergy shots),766908.0


In [0]:
dbutils.notebook.exit("Success")